In [ ]:
#根据股票收盘价格，计算股票之间的相关系数，0为无关，1为正相关，-1为负相关。
    
    
import time

def main_data_corr():
    d_time = time.time() #用作计算程序运行耗时
    
    sample_amount = 3000 #定义计算多少个自然天的数据
    
    fields = 'close' #定义统计的元素。例如'open'为开盘价，'close'为收盘价。参考tushare中daily方法的参数。

    ty = int(input("请输入股票类型，0表示主板和中小板，1表示上交所主板，2表示深交所主板，3表示深交所中小板：")) #输入股票类型  
    
    #获得股票代码
    stock_code = acq_code(ty)
    
    end_date = datetime.datetime.now() #默认取最新的数据
    end_date = end_date.strftime('%Y%m%d')
    end_date = '20191101' #如果希望获取之前的数据，直接在此处赋值截至日期

    #获得股票行情数据
    sts_data = acq_data(stock_code, end_date, sample_amount, fields) 

    #计算相关系数，并保存到文件
    fn = cal_corr(stock_code,sts_data) 
    
    print('任务完成！相关系数结果保存为文件：', fn, '总耗时：',time.strftime('%H:%M:%S',time.localtime(time.time()-d_time)))



import tushare as ts

def acq_code(ty):
    #首先获得全部的股票代码
    pro = ts.pro_api('f3bbc97d0ffbbed8666e6f7c82e712165950d048987f5d6cfbf1e0ce') #''的token可以到“https://tushare.pro/”注册获得
    pool = pro.stock_basic(exchange = '',list_status = 'L',adj = 'qfq',fields = 'ts_code')
    print('成功获取全部', len(pool), '只股票代码……')

    #筛选股票
    stock_code = []
    k = 1
    for i in pool.ts_code:
        if k==3: #在调试时，减少数据量。正常运行时关掉该判断（该行和下一行）
            break
        if ty == 1:
            if i[0:2]=='60': #上交所主板 
                stock_code.append(i)
                k = k + 1
        elif ty == 2: #深交所主板
            if i[0:3]=='000': #上交所主板 
                stock_code.append(i)
                k = k + 1
        elif ty == 3:
            if i[0:3]=='002': #上交所主板 
                stock_code.append(i)
                k = k + 1
        elif ty == 0:
            if (i[0:2]=='00' or i[0:2]=='60'): #只保留主板和中小板的股票
                stock_code.append(i)
                k = k + 1

    print('完成筛选，只保留主板和中小板的股票共：', len(stock_code), '只股票代码……')

    return stock_code



import tushare as ts
import datetime

def acq_data(stock_code, end_date, sample_amount, fields):
    print('开始获取股票行情数据，这将花一些时间……')
    
    #初始化数据
    sts_data = [[]for i in range(len(stock_code))] #定义数组。

    #整理开始和结束时间
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days = sample_amount) #计算数据的天数
    end_date = end_date.strftime('%Y%m%d')
    start_date = start_date.strftime('%Y%m%d')

    #定义数据接口的token
    pro = ts.pro_api('f3bbc97d0ffbbed8666e6f7c82e712165950d048987f5d6cfbf1e0ce') #token可以到“https://tushare.pro/”注册获得
   
    for i in range(len(stock_code)):
    #从数据接口获取数据
        try:
            stock_data = pro.query('daily', ts_code = stock_code[i], start_date = start_date, end_date = end_date, fields = fields) #获取该股票历史数据
        except:
            print('连接超时，等待60秒再继续……') #普通用户，tushare每分钟只能读取200次数据。
            time.sleep(60)
            stock_data = pro.query('daily', ts_code = stock_code[i], start_date = start_date, end_date = end_date, fields = fields) #获取该股票历史数据
    
    #进度提示
        if int(i/100) == (i/100) and i > 0: 
            print('已获取：',i,'只股票数据，继续获取……')

        sts_data[i] = stock_data.close.tolist()

    return sts_data



import pandas as pd

def cal_corr(stock_code, sts_data):
    print('正在计算相关系数……')
    df = {}
    for i in range(len(stock_code)):
        df[stock_code[i]] = sts_data[i] #将股票数据存入字典   

    df_corr=pd.DataFrame(dict([(k, pd.Series(v)) for k, v in df.items()])) #字典转为dataframe格式，以便使用corr函数。由于数据不等长，需处理。
    
    cal_corr = df_corr.corr() #计算相关系数

    fn = 'data_corr.csv'
    cal_corr.to_csv(fn) #保存到文件
    
    return fn



if __name__ == '__main__': #运行程序
    main_data_corr()


In [ ]:
__name__